# DataFrame 마무리 & Covariance Matrix(1)
- - -

#### 1. Performing a `Map` command in DataFrame
* In order to perform a map on a dataframe, you first need to transform it into an RDD!
* Not the recommended way. Better to use built-in sparkSQL functions

<br>

#### 2. Covariance Matrix

* Calculating the mean of Sample Vectors

* Outer product of sample vectors

* Covariance Matrix
- - -

### 1. Performing a `Map` command in DataFrame

#### pyspark import & SparkContext 생성

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType

sc = SparkContext(master="local[*]")
print(sc)

# Just like using Spark requires having a SparkContext, using SQL requires an SQLContext
sqlContext = SQLContext(sc)
print(sqlContext)

<SparkContext master=local[*] appName=pyspark-shell>


In [2]:
!pip install GoogleDriveDownloader

In [3]:
#### 예제 파일 다운로드
from os.path import exists
from google_drive_downloader import GoogleDriveDownloader as gdd
import tarfile

if exists("./NY.tgz"):
    !rm -rf ./NY.tgz
if exists("./NY.parquet"):
    !rm -rf ./NY.parquet
    
gdd.download_file_from_google_drive(file_id='1hAHV6vC6FvVgrYnoN-lR-IfH488-H121',
                                   dest_path = './NY.tgz')
!tar -xzvf NY.tgz
df = sqlContext.read.load("NY.parquet")

NY.parquet/
NY.parquet/_SUCCESS
NY.parquet/part-00022-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00000-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00021-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00001-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00023-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00002-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00024-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00003-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00025-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00004-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00027-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-00005-89caf7c0-9733-40ec-a650-7f368529dd01-c000.snappy.parquet
NY.parquet/part-0000

In [5]:
# printSchema()와 sample를 이용한 데이터 확인
df.printSchema()
# sample 사용법 참조
# https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.sample
df.sample(False, 0.01).show(3)

root
 |-- Station: string (nullable = true)
 |-- Measurement: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- Values: binary (nullable = true)
 |-- dist_coast: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- elevation: double (nullable = true)
 |-- state: string (nullable = true)
 |-- name: string (nullable = true)

+-----------+-----------+----+--------------------+------------------+-----------------+------------------+------------------+-----+-----------------+
|    Station|Measurement|Year|              Values|        dist_coast|         latitude|         longitude|         elevation|state|             name|
+-----------+-----------+----+--------------------+------------------+-----------------+------------------+------------------+-----+-----------------+
|USW00014743|   TMIN_s20|1944|[D6 D7 D6 D7 D5 D...| 388.9079895019531|44.57720184326172|-75.10970306396484|136.60000610351562|   NY|      CANTON 4 SE|

* DataFrame to RDD : `[DataFrame].rdd`, 각각의 요소가 `Row`인 RDD 생성

* RDD to DataFrame : `sqlContext.createDataFrame([RDD], schema)`

단, RDD에서 DataFrame으로 변환시, schema를 꼭 정의해줘야 한다!

In [5]:
some_rdd = df.rdd.takeSample(False, 1)
some_rdd

[Row(Station='USC00308058', Measurement='TMIN_s20', Year=1963, Values=bytearray(b'\x06\xd7\x1e\xd76\xd7N\xd7d\xd7y\xd7\x91\xd7\xaa\xd7\xc0\xd7\xd6\xd7\xec\xd7\x01\xd8\x0b\xd8\x14\xd8\x1d\xd8&\xd8/\xd87\xd8?\xd8G\xd8O\xd8U\xd8\\\xd8b\xd8h\xd8m\xd8p\xd8t\xd8v\xd8x\xd8y\xd8y\xd8x\xd8v\xd8s\xd8p\xd8l\xd8h\xd8c\xd8]\xd8U\xd8M\xd8D\xd89\xd8/\xd8%\xd8\x19\xd8\r\xd8\xff\xd7\xe3\xd7\xc5\xd7\xa6\xd7\x86\xd7f\xd7F\xd7#\xd7\xff\xd6\xd9\xd6\xb3\xd6\x8c\xd6d\xd6;\xd6\x12\xd6\xe8\xd5\xbc\xd5\x90\xd5c\xd56\xd5\n\xd5\xdd\xd4\xb2\xd4\x86\xd4Y\xd4-\xd4\xfd\xd3\xa4\xd3M\xd3\xf5\xd2\x9e\xd2H\xd2\xf5\xd1\xa6\xd1V\xd1\x07\xd1\xb9\xd0m\xd0$\xd0\xba\xcf0\xcf\xa4\xce\x18\xce\x93\xcd\x19\xcd\xa0\xcc)\xccg\xcb~\xca\x92\xc9\xa2\xc8\x95\xc7\xfd\xc5f\xc4y\xc1;\xbc\xdf8S@oCdE\x02GOH\x15I\xd8I\x8eJ>K\xf7KXL\xb3L\x14MvM\xd7M?N\xa4N\x02OgO\xccO\x19PLP\x81P\xb7P\xeeP%Q]Q\x96Q\xcdQ\x00R6RkR\xa1R\xd9R\rSCSxS\xacS\xdfS\nT&TAT[TwT\x91T\xabT\xc4T\xddT\xf6T\x0eU&U=UTUjU\x80U\x95U\xa9U\xbeU\xd3U\xe7U\xfcU\x0fV"V5VGVXViV{V\x8cV\

* Lec 6의 예제

    1. 주어진 DataFrame에서 `Year가 1900 미만인 경우 '19th'`, `2000 미만인 경우 '20th'`, `2010 미만인 경우 '21st'`, `모두 아닐 경우 'possibly_bad_data'로 값을 치환`하여라

 여기서 `map`에 들어가는 `input`이 무엇인지 반드시 숙지하여야 한다.
    


In [7]:
def find_century(row):
    if row.Year < 1900:
        return "19th"
    elif row.Year < 2000:
        return "20th"
    elif row.Year < 2010:
        return "21st"
    else:
        return "possibly_bad_data"
    
df.rdd.map(find_century).take(5)

['20th', '20th', '20th', '20th', '20th']

* Lec 6의 예제

    2. 주어진 DataFrame에서 각각의 요소 중 `longitude`와 `latitude`를 추출하여 (longitude, latitude)의 형태로 값을 출력하여라.

In [8]:
df.rdd.map(lambda row : (row.longitude, row.latitude)).take(5)

[(-77.71330261230469, 42.57080078125),
 (-77.71330261230469, 42.57080078125),
 (-77.71330261230469, 42.57080078125),
 (-77.71330261230469, 42.57080078125),
 (-77.71330261230469, 42.57080078125)]

### Excercise 1 - RDD function in DataFrame (60 point)

- - -
task 별로 ``위에서 사용한 DataFrame(df)``에서 ``RDD``로 변환 후 ``RDD function``을 적용하여 해결합니다

(task 당 20 point)

---

**task**

* 1 : ``df``에서 ``name``별 가장 최근 ``Year``를 **map과 reduce, 또는 reduceByKey 등**을 활용하여 구한 후, take(10)을 통하여 출력합니다.(20 point)

<br>

* 2 : ``df``에서 ``Year``가 ``2000 이상``인 결과에 대해, ``name``별 ``Year``, ``dist_coast``, ``elevation``의 평균을 구하고 ``name``를 기준으로 정렬(Z->A)한 후, take(5)을 통하여 출력합니다.(20 point)

<br>

* 3 : ``df``에서 ``Measurement`` 별 `Values`의 1일부터 10일까지(``np.frombuffer(row.Values[:20], dtype='float16')`` 또는 ``np.frombuffer(row.Values, dtype = 'float16')[:10])의 합이 가장 큰 ``Year``와 ``그 값을 구한 후``, ``Measurement``를 기준으로 정렬(A->Z)합니다. 마지막으로 collect를 하여 출력합니다.(20 point) 

- - -

**task**

* 1 : ``df``에서 ``name``별 가장 최근 ``Year``를 **map과 reduce, 또는 reduceByKey 등**을 활용하여 구한 후, take(10)을 통하여 출력합니다.(20 point)

**★ DataFrame이 아닌 RDD로 작업할 것**

```
#task1 output
[('DANSVILLE MUNI AP', 2013),
 ('BRIDGEHAMPTON', 2013),
 ('MIDDLETOWN 2 NW', 2011),
 ('BERLIN 5 S', 2000),
 ('ELMIRA CORNING RGNL AP', 2013),
 ('UNADILLA 2 N', 2013),
 ('SUFFERN 2 E', 1955),
 ('ROXBURY', 1972),
 ('LOWVILLE', 2013),
 ('GABRIELS', 1978)]

```

In [6]:
# 1-1 답 작성  
df.rdd.map(lambda row:(row.name,row.Year)).reduceByKey(lambda x,y:x if x>y else y).take(10)#df에서 rdd로 변환해주는데 row의name열과 year만 뽑아서 가져오고 그 후에 키 값을 묶고 value가 긴 값을 뽑아냄


[('SABATTIS WHITNEY PARK', 1958),
 ('RAY BROOK', 2004),
 ('SHARON SPRINGS 1N', 1953),
 ('MIDDLETOWN 2 NW', 2011),
 ('BERLIN 5 S', 2000),
 ('ELMIRA CORNING RGNL AP', 2013),
 ('WALTON', 1997),
 ('GRAFTON', 2004),
 ('WALDEN 1 ESE', 2012),
 ('GARNERVILLE', 1989)]

**task**

* 2 : ``df``에서 ``Year``가 ``2000 이상``인 결과에 대해, ``name``별 ``Year``, ``dist_coast``, ``elevation``의 평균을 구하고 ``name``를 기준으로 정렬(Z->A)한 후, take(10)을 통하여 출력합니다.(20 point)

**★ DataFrame이 아닌 RDD로 작업할 것**

```
# task2 output
[('YOUNGSTOWN 2 NE', (2008.5, 476.80999755859375, 85.30000305175781)),
 ('YORKTOWN HEIGHTS 1W',
  (2006.421686746988, 28.945999145507812, 204.1999969482422)),
 ('WINDHAM 3 E', (2004.8387096774193, 147.28700256347656, 512.0999755859375)),
 ('WILLSBORO 1 N',
  (2005.360655737705, 255.24200439453125, 54.900001525878906)),
 ('WHITNEY POINT DAM', (2009.2826086956522, 235.35800170898438, 317.0))]
 ```

In [7]:
a=(1,2)
b=(2,3)
print(a[0]/b[1])

0.3333333333333333


In [8]:
# 1-2 답 작성  
a=df.rdd.map(lambda row:(row.name,(row.Year,row.dist_coast,row.elevation,1))).filter(lambda x:(x[1][0])>=2000).reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1],x[2]+y[2],x[3]+y[3]))
a.map(lambda row:(row[0],(row[1][0]/row[1][3],row[1][1],row[1][3],row[1][2]/row[1][3]))).take(10)
#a에 df을 rdd형태로 바꿔주고 row형식으로 나타내는데 row는 key,value RDD형태로 만들어주는데 key는 row.name,value는 row.year,row.dist_coast,row.elevtion,1로 만들어주고 row.year이 2000년 이상으로 조건을 걸어주고  reduceByKey를 통해 ,x와 y의  각 자리의 값을 더해줌
#a를 매핑해서 나온 결과를 나눠서 원하는 값을 얻고 10개만 값을 봄


[('RAY BROOK',
  (2001.8148148148148, 16392.617248535156, 54, 493.79998779296875)),
 ('MIDDLETOWN 2 NW', (2005.5, 11675.332397460938, 144, 213.39999389648438)),
 ('BERLIN 5 S', (2000.0, 904.9979553222656, 6, 347.5)),
 ('ELMIRA CORNING RGNL AP',
  (2005.78, 28038.198852539062, 100, 291.1000061035156)),
 ('GRAFTON', (2002.0, 10232.279663085938, 60, 475.5)),
 ('WALDEN 1 ESE', (2006.0, 11223.918640136719, 156, 115.80000305175781)),
 ('TRENTON FALLS',
  (2007.4705882352941, 28717.793701171875, 102, 243.8000030517578)),
 ('PAVILION',
  (2008.2631578947369, 46388.310974121094, 114, 291.3999938964844)),
 ('MASSENA INTL AP',
  (2006.7291666666667, 33609.791015625, 96, 65.19999694824219)),
 ('LIBERTY 1 NE', (2006.0, 20039.603942871094, 156, 481.6000061035156))]

**task**

* 3 : ``df``에서 ``Measurement`` 별 `Values`의 1일부터 10일까지(``np.frombuffer(row.Values[:20], dtype='float16')`` 또는 ``np.frombuffer(row.Values, dtype = 'float16')[:10])의 합이 가장 큰 ``Year``와 ``그 값을 구한 후``, ``Measurement``를 기준으로 정렬(A->Z)합니다. 마지막으로 collect를 하여 출력합니다.(20 point) 

<br>

**★ DataFrame이 아닌 RDD로 작업할 것**

★★여기서 ``Values``는 ``bytearray`` type입니다. ★★

``numpy``의 **frombuffer**를 이용하여 ``float16``으로 바꿉니다.
자세한 사용법은 [여기](https://docs.scipy.org/doc/numpy/reference/generated/numpy.frombuffer.html)를 참고합니다

★★또한, ``numpy``의 ``nansum``을 이용하여 값이 **nan**이 아닌 Values의 합을 구합니다.★★

```
# task3 output
[('PRCP', (12824.0, 1946)),
 ('PRCP_s20', (4264.0, 1983)),
 ('SNOW', (2912.0, 1954)),
 ('SNOW_s20', (4140.0, 1895)),
 ('SNWD', (16590.0, 1970)),
 ('SNWD_s20', (12320.0, 1976)),
 ('TMAX', (2216.0, 1897)),
 ('TMAX_s20', (2050.0, 1897)),
 ('TMIN', (623.0, 2007)),
 ('TMIN_s20', (21360.0, 1987)),
 ('TOBS', (1000.0, 1998)),
 ('TOBS_s20', (969.5, 1992))]
 ```

In [22]:
import numpy as np
a=df.rdd.map(lambda row:(row.Measurement,(np.nansum(np.frombuffer(row.Values, dtype = 'float16')[:10]),row.Year))).reduceByKey(lambda x,y:x if x[0]>y[0] else y)
a.collect()
#a는 rdd형태로 만들고 map을 통해서 row를 만들어주는데 key는 Measurement로 하고 nansum함수를 이용하여 row.values를 dtype='float16'으로 바꾼후 10까지 더하고뒤에 row.year을 넣어서 row를 마무리 후 rreduceyke를 통해 x[0]와y[0] 합의 값을 비교 후 큰 것을 산출
    

[('PRCP', (12824.0, 1946)),
 ('TMAX', (2216.0, 1897)),
 ('TOBS', (1000.0, 1998)),
 ('SNWD_s20', (12320.0, 1976)),
 ('SNOW_s20', (4140.0, 1895)),
 ('PRCP_s20', (4264.0, 1983)),
 ('TOBS_s20', (969.5, 1992)),
 ('TMIN_s20', (21360.0, 1987)),
 ('TMIN', (623.0, 2007)),
 ('TMAX_s20', (2050.0, 1897)),
 ('SNOW', (2912.0, 1954)),
 ('SNWD', (16590.0, 1970))]

In [147]:
import numpy as np
a=df.rdd.map(lambda row:(row.Measurement,np.nansum(np.frombuffer(row.Values, dtype = 'float16')[:10])))
a.collect()                                                                                                          


# 1-3 답 작성


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:37259)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:37259)

### 2. Spark와 Numpy를 사용하여 Covariance Matrix 구해보자!

#### (1) numpy 기초 
아래 참고자료를 활용하여 numpy 기초 학습

* 참고자료 
  * [참고자료 1](http://taewan.kim/post/numpy_cheat_sheet/)
  * [참고자료 2](https://docs.scipy.org/doc/numpy/user/quickstart.html)
  * [참고자료 3](https://scipy-lectures.org/intro/numpy/array_object.html#what-are-numpy-and-numpy-arrays)
  * [참고자료 4](https://doorbw.tistory.com/171)
  * [참고자료 5](https://datascienceschool.net/view-notebook/17608f897087478bbeac096438c716f6/)

* 위 자료에서중에서도 
  * ndarray 생성법
  * vector, matrix 연산
  * 인덱싱 (slicing)
  * 행렬 합치기 (vstack, dstack, hstack)
  * sum, mean
  * np.nan 자료형
  * reshape
  * matmul


#### (2)  Calculating the mean of Sample Vectors 
다음 벡터들을 샘플 벡터로 가정합니다.
* $n$ 은 샘플의 수 이고
* $d$는 각 데이터 벡터의 길이 입니다 (예를 들어서 날씨데이터의 경우 $d=365$)
$$
\mathbf{x}_i = \left[\begin{array}{cccc}
x_{i1} & x_{i2}& \ldots & x_{id}, 
\end{array}\right], \quad i=1,\ldots, n
$$
Sample vector 들의 mean (평균) 벡터 $\bar{\mathbf{x}}$는 다음과 같이 구합니다 
$$
\bar{\mathbf{x}} = \frac{1}{n}\sum_{i=1}^n \mathbf{x}_i
$$


In [0]:
# 평균 벡터 구하는 문제 (sum, mean 활용 둘다 해도 괜찮습니다))

In [6]:
# in python
import numpy as np

sample1 = np.array([1,2,3])
sample2 = np.array([4,5,6])
sample3 = np.array([7,8,9])
print("각 sample의 값 : ", sample1, sample2, sample3)
print("sample vector sum : ", sample1 + sample2 + sample3)
print("sample vector mean : ", (sample1 + sample2 + sample3) / 3)

각 sample의 값 :  [1 2 3] [4 5 6] [7 8 9]
sample vector sum :  [12 15 18]
sample vector mean :  [4. 5. 6.]


In [7]:
# in spark
vector_list = sc.parallelize([np.array([1,2,3]),np.array([4,5,6]),np.array([7,8,9])])
print("각 sample의 값 : ", vector_list.collect())
print("sample vector sum : ", vector_list.reduce(lambda ndarr1, ndarr2 : ndarr1 + ndarr2))
print("sample vector mean : ",
      vector_list.reduce(lambda ndarr1, ndarr2 : ndarr1 + ndarr2) / vector_list.count())

각 sample의 값 :  [array([1, 2, 3]), array([4, 5, 6]), array([7, 8, 9])]
sample vector sum :  [12 15 18]
sample vector mean :  [4. 5. 6.]


### Excercise 2 - Calculating the mean of Sample Vectors (40 point)

- - -
 
다음 데이터에 대하여 다음 과제를 수행하세요.

- regular.csv : KBO에서 활약한 타자들의 역대 정규시즌 성적을 포함하여 몸무게, 키 ,생년월일 등의 기본정보

**위의 두 데이터는 모두 `,`로 구분되어 있습니다.**

 - **데이터의 자세한 설명은 다음의 링크를 참조해주세요.([여기를 눌러서 12. 데이터 설명 참고](https://dacon.io/cpt6/62885))**
 - 또한 regular.csv를 직접 열어서 데이터가 어떻게 저장되어 있는지 확인해주세요.

- - -

**task**

- 1. filter를 사용하여 팀 이름이 ``두산``인 선수에 대해, ``(batter_id, np.array([G,R,H,RBI,BB]))``의 형태로 Key/Value RDD를 생성합니다. (20 point)

    1. G R H RBI BB의 경우 초기 설정값이 ``stirng``.  이 값들을 ``float64``으로 변경할 것. ex) ``np.array([1,2,3], dtype = 'float64')``
    2. ★ 각 값이 `' '`일 경우, 0 으로 변경할 것. 
    3. ``map``에서 바로 적용 또는 그러한 함수를 작성
    

<br>

- 2. (20 point)
    1. ``reduceByKey``를 사용하여 ``batter_id``(Key)가 동일한 선수의 ``G, R, H, RBI, BB``(Value)를 각각 더해준 후 ``batter_id``(Key)를 기준으로 ``sortByKey``를 적용합니다. 그 후, ``map``을 사용하여 ``G, R, H, RBI, BB``(Value)만 선택 후 새로운 RDD로 만듭니다.
    2. 위에서 생성된 RDD에 대해, 중복되지 않는 sample의 수를 ``count``를 이용하여 구합니다.
    3. ``reduce``를 이용하여 ``G, R, H, RBI, BB``(Value)를 모두 더한 후 위에서 구한 sample의 수(`count`)로 나누어서 sample vector의 평균을 구합니다.



---

In [6]:
import urllib.request
import re
import numpy as np

f = urllib.request.urlretrieve ("https://docs.google.com/uc?export=download&id=1b_L-rJYJC9Oqga0fQ2zh2M763CTM8jzR", "regular.csv")
regular = sc.textFile("./regular.csv").map(lambda x : x.split(",")) #RDD 형태

**task**

- 1. filter를 사용하여 팀 이름이 ``두산``인 선수에 대해, ``(batter_id, np.array([G,R,H,RBI,BB]))``의 형태로 Key/Value RDD를 생성합니다. (20 point)

    1. G R H RBI BB의 경우 초기 설정값이 ``stirng``.  이 값들을 ``float64``으로 변경할 것. ex) ``np.array([1,2,3], dtype = 'float64')``
    2. ★ 각 값이 `' '`일 경우, 0 으로 변경할 것. 
    3. ``map``에서 바로 적용 또는 그러한 함수를 작성
    
```
#output
[(7, array([1., 0., 0., 0., 0.])),
 (7, array([64., 16., 21., 11.,  6.])),
 (7, array([93., 30., 40., 25., 16.])),
 (7, array([6., 3., 3., 3., 2.])),
 (7, array([40., 14., 17.,  5., 10.])),
 (7, array([48., 12., 24., 10., 18.])),
 (17, array([16.,  1.,  1.,  1.,  0.])),
 (17, array([32.,  2.,  3.,  0.,  0.])),
 (17, array([16.,  2.,  2.,  0.,  0.])),
 (17, array([116.,  38.,  85.,  29.,  24.]))]
 ```

In [53]:
a=regular.filter(lambda x:x[3]=='두산').map(lambda x:(x[0],np.frombuffer(np.array([x[5],x[7],x[8],x[13],x[16]],dtype='float64'))))
a.collect()
#regular가 rdd형태이므로 filter를 통해 두산인 선수만 거르고 map을 통해 x[0]인 batter_id를 key값 value값에는 주어진 조건에 넣고 dtyper을 float64fh 바꾼후 출력

                    

[('7', array([1., 0., 0., 0., 0.])),
 ('7', array([64., 16., 21., 11.,  6.])),
 ('7', array([93., 30., 40., 25., 16.])),
 ('7', array([6., 3., 3., 3., 2.])),
 ('7', array([40., 14., 17.,  5., 10.])),
 ('7', array([48., 12., 24., 10., 18.])),
 ('17', array([16.,  1.,  1.,  1.,  0.])),
 ('17', array([32.,  2.,  3.,  0.,  0.])),
 ('17', array([16.,  2.,  2.,  0.,  0.])),
 ('17', array([116.,  38.,  85.,  29.,  24.])),
 ('17', array([126.,  89., 119.,  66.,  61.])),
 ('17', array([126.,  84., 114.,  70.,  74.])),
 ('17', array([85., 59., 66., 29., 33.])),
 ('17', array([100.,  53.,  60.,  35.,  34.])),
 ('17', array([93., 31., 37., 16., 18.])),
 ('17', array([58., 33., 40., 26., 13.])),
 ('17', array([10.,  3.,  4.,  1.,  3.])),
 ('17', array([52., 18., 27.,  7., 11.])),
 ('17', array([41., 13., 22., 11.,  6.])),
 ('17', array([8., 1., 1., 1., 1.])),
 ('20', array([3., 0., 0., 0., 0.])),
 ('20', array([11.,  6.,  4.,  5.,  2.])),
 ('20', array([58., 28., 42., 24., 21.])),
 ('20', array([57

**task**

- 2. (20 point)
    1. ``reduceByKey``를 사용하여 ``batter_id``(Key)가 동일한 선수의 ``G, R, H, RBI, BB``(Value)를 각각 더해준 후 ``batter_id``(Key)를 기준으로 ``sortByKey``를 적용합니다. 그 후, ``map``을 사용하여 ``G, R, H, RBI, BB``(Value)만 선택 후 새로운 RDD로 만듭니다.
    2. 위에서 생성된 RDD에 대해, 중복되지 않는 sample의 수를 ``count``를 이용하여 구합니다.
    3. ``reduce``를 이용하여 ``G, R, H, RBI, BB``(Value)를 모두 더한 후 위에서 구한 sample의 수(`count`)로 나누어서 sample vector의 평균을 구합니다.

```
# output
[G, R, H, RBI, BB] mean :  [427.8846  186.32692 344.05768 178.0577  129.48077]
```

In [97]:
a=1,2]
b=(2,3)
a.reduce(lambda x:x*2)

AttributeError: 'list' object has no attribute 'reduce'

In [101]:
a=regular.filter(lambda x:x[3]=='두산').map(lambda x:(x[0],np.frombuffer(np.array([x[5],x[7],x[8],x[13],x[16]],dtype='float64')))).reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1],x[2]+y[2],x[3]+y[3],x[4]+y[4]))
#regular가 rdd형태이므로 filter를 통해 두산인 선수만 거르고 map을 통해 x[0]인 batter_id를 key값 value값에는 주어진 조건에 넣고 dtyper을 float64fh 바꾼후 출력후 reduceByKey를 통해 원소의 각 값을 더해줌
a.sortByKey()
#나온 값을 정렬
b=a.map(lambda x:(x[1][0],x[1][1],x[1][2],x[1][3],x[1][4]))#뒤의 g,r,h,rbi,bb등의 원소만 rdd로 출력
c=b.count()#원소의 갯수를 카운트하여 c에 저장
task2_mean_vector=b.reduce(lambda x,y:((x[0]+y[0]/c),(x[1]+y[1]/c),(x[2]+y[2]/c),(x[3]+y[3]/c),(x[4]+y[4]/c)))
# task2_mean_vector에 b를 reduce 하여 g,r,h,rbi,bb의 값을 더하고 count로 나눈 후 저장

#b는 원소의 값만을 rdd로만 뽑아냄
#b.collect()
#output
print("[G, R, H, RBI, BB] mean : ", task2_mean_vector) #결과를 출력

[G, R, H, RBI, BB] mean :  (1143.6386834319528, 538.2910502958581, 789.4400887573967, 398.0329142011835, 356.64201183431965)


# 기말고사가 다가온다.ㅋ